In [1]:
import os
import openai
from typing import Any, List
#from langchain import PromptTemplate
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from azure.ai.generative.index import get_langchain_retriever_from_index

from langchain.retrievers import AzureCognitiveSearchRetriever
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.vectorstores import FAISS

Set credential values needed to authenciate and authorize access to your Azure OpenAI instance.

In [2]:
def setup_credentials():
    # Azure OpenAI credentials
    openai.api_type = os.environ["OPENAI_API_TYPE"]
    openai.api_key = os.environ["OPENAI_API_KEY"]
    openai.api_version = os.environ["OPENAI_API_VERSION"]
    openai.api_base = os.environ["OPENAI_API_BASE"]

    # Azure Cognitive Search credentials
    os.environ["AZURE_COGNITIVE_SEARCH_TARGET"] = os.environ["AZURE_AI_SEARCH_ENDPOINT"]
    os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"] = os.environ["AZURE_AI_SEARCH_KEY"]
    os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"] = "search-ai-aiprog-hub" #"AzureAISearch"
    os.environ["AZURE_COGNITIVE_SEARCH_INDEX_NAME"] = os.environ["AZURE_AI_SEARCH_INDEX_NAME"]

Execute the authorize with the given credentials above

In [3]:
setup_credentials()

In [5]:
from langchain.retrievers import AzureCognitiveSearchRetriever
retriever = AzureCognitiveSearchRetriever(content_key="content", top_k=10)

Set you cognitive search endpoint and key for the vector store

Define the AzureSearch instance with the OpenAIEmbeddings to vectorize the input sample text data

Construct your prompt by specifying what the chat assistant does as well as the scope and domain of topics it can provide information for.  In addition, define any constraints, restricts or bounderies on how the prompt show behave.  The prompt template includes action the prompt can take; chat history of the system/user dialogue; context of the chat conversation.

In [10]:
template = """
System:
You are an AI assistant helping users with queries related to outdoor outdooor/camping gear and clothing.
Use the following pieces of context to answer the questions about outdoor/camping gear and clothing as completely, correctly, and concisely as possible.

---

{chat_history}

---


Question: {question}

Answer:"
"""

prompt_template = PromptTemplate(
    template=template,
    input_variables=[
        "context",
        "chat_history",
        "question"
    ],
)

In [11]:
memory = ConversationBufferMemory(memory_key="chat_history")

Initialize the AzureChatOpenAI instance with your Azure Open AI model and deployment.  The temperature value ranges from 0 to 1.  Value close to 0 denote how specific you want the response to be and value close to 1 denote how randomly you want the responses to be.

In [12]:
llm = AzureChatOpenAI(
    deployment_name=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT"],
    model_name=os.environ["AZURE_OPENAI_CHAT_MODEL"],
    temperature=0.7
)

Now, you can ready to invoke the key of langchain you want to use.  <provide the type of conversion you want>.  To activate the instance you need your LLM model to retrieve response, the promt template including the rules, and verbose <???>

In [13]:
# convert MLIndex to a langchain retriever
search_index_folder = os.getenv("AZURE_AI_SEARCH_INDEX_NAME") + "-mlindex"
index_langchain_retriever = get_langchain_retriever_from_index(search_index_folder)

print(search_index_folder)
print(index_langchain_retriever)

"""

qna_chain =RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=index_langchain_retriever,
    #retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": prompt_template,
    }
    
)
"""

ActivityStarted, MLIndex.__init__
ActivityCompleted: Activity=MLIndex.__init__, HowEnded=Success, Duration=25.11 [ms]
ActivityStarted, MLIndex.as_langchain_vectorstore
azure-search-documents==11.4.0b11 not compatible langchain.vectorstores.azuresearch yet, using REST client based VectorStore.
ActivityCompleted: Activity=MLIndex.as_langchain_vectorstore, HowEnded=Success, Duration=38.82 [ms]


/workspaces/aistudio-copilot-sample/product-info-mlindex
tags=['AzureCognitiveSearchVectorStore'] vectorstore=<azure.ai.generative.index._langchain.acs.AzureCognitiveSearchVectorStore object at 0x7f612cec34f0>


'\n\nqna_chain =RetrievalQA.from_chain_type(\n    llm=llm,\n    chain_type="stuff",\n    retriever=index_langchain_retriever,\n    #retriever=retriever,\n    return_source_documents=True,\n    chain_type_kwargs={\n        "prompt": prompt_template,\n    }\n    \n)\n'

In [14]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [15]:
vectorstore = index_langchain_retriever
print(vectorstore)

tags=['AzureCognitiveSearchVectorStore'] vectorstore=<azure.ai.generative.index._langchain.acs.AzureCognitiveSearchVectorStore object at 0x7f612cec34f0>


In [16]:
qna_chain = ConversationalRetrievalChain.from_llm(llm=llm,
                                           retriever=retriever,
                                           condense_question_prompt=prompt_template,
                                           #return_source_documents=True,
                                           verbose=False,
                                           memory=memory)

Let's test the chat with a question!

In [17]:
#when you want to initalize chat with history.



inquiry = "which tent is the most waterproof?"
response = qna_chain({"question": inquiry})
response['answer']

'Based on the available information, both the Alpine Explorer Tent (item_number: 8) and the SkyView 2-Person Tent (item_number: 15) are described as waterproof in the reviews. However, there is no specific comparison provided to determine which tent is more waterproof.'